repo : https://github.com/Azure/lstms_for_predictive_maintenance

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation

from sklearn.metrics import confusion_matrix, recall_score, precision_score

In [2]:
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
telemetry = pd.read_csv('./data/PdM_telemetry.csv')
errors = pd.read_csv('./data/PdM_errors.csv')
maint = pd.read_csv('./data/PdM_maint.csv')
failures = pd.read_csv('./data/PdM_failures.csv')
machines = pd.read_csv('./data/PdM_machines.csv')

In [4]:
telemetry['datetime'] = pd.to_datetime(telemetry['datetime'], format="%Y-%m-%d %H:%M:%S")

errors['datetime'] = pd.to_datetime(errors['datetime'],format = '%Y-%m-%d %H:%M:%S')
errors['errorID'] = errors['errorID'].astype('category')

maint['datetime'] = pd.to_datetime(maint['datetime'], format='%Y-%m-%d %H:%M:%S')
maint['comp'] = maint['comp'].astype('category')

machines['model'] = machines['model'].astype('category')

failures['datetime'] = pd.to_datetime(failures['datetime'], format="%Y-%m-%d %H:%M:%S")
failures['failure'] = failures['failure'].astype('category')

In [5]:
# create a column for each error type
# error 별 column 생성
error_count = pd.get_dummies(errors.set_index('datetime')).reset_index()
error_count.columns = ['datetime', 'machineID', 'error1', 'error2', 'error3', 'error4', 'error5']
# 같은 시간으로 통합
error_count = error_count.groupby(['machineID','datetime']).sum().reset_index()
# telemetry 에 통합
error_count = telemetry[['datetime', 'machineID']].merge(error_count, on=['machineID', 'datetime'], how='left').fillna(0.0)
error_count = error_count.dropna()

In [6]:
# create a column for each error type
# 각 부품별로 생성후 column 명 설정
comp_rep = pd.get_dummies(maint.set_index('datetime')).reset_index()
comp_rep.columns = ['datetime', 'machineID', 'comp1', 'comp2', 'comp3', 'comp4']

# combine repairs for a given machine in a given hour
# 기계와 시간별로 교체 부품 정리
comp_rep = comp_rep.groupby(['machineID', 'datetime']).sum().reset_index()

# add timepoints where no components were replaced
# telemetry 데이터에 데이터 결합
comp_rep = telemetry[['datetime', 'machineID']].merge(comp_rep,
                                                      on=['datetime', 'machineID'],
                                                      how='outer').fillna(0).sort_values(by=['machineID', 'datetime'])

components = ['comp1', 'comp2', 'comp3', 'comp4']
for comp in components:
    # convert indicator to most recent date of component change
    comp_rep.loc[comp_rep[comp] < 1, comp] = None # 0.0인 위치를 모두 None 으로 변경
    comp_rep.loc[-comp_rep[comp].isnull(), comp] = comp_rep.loc[-comp_rep[comp].isnull(), 'datetime']
    # -comp_rep[comp].isnull() comp_req 의 comp columns 에서 None 이 아닌 부분을 선택(즉 해당 부품의 교체가 발생된 날짜 선택)
    # 해당 부분의 내용을 시간으로 변경
    
    # forward-fill the most-recent date of component change
    # 나머지 빈 부분을 모두 해당 시간으로 교체 (즉 부품 교체일로 변경 날짜)
    comp_rep[comp] = comp_rep[comp].fillna(method='ffill')

# remove dates in 2014 (may have NaN or future component change dates)    
# 2014년 이상의 데이터만 선택
comp_rep = comp_rep.loc[comp_rep['datetime'] > pd.to_datetime('2015-01-01')]

# replace dates of most recent component change with days since most recent component change
# 각 comp 내용을 부품 교체일 부터 얼마나 지났는지로 변경
for comp in components:
    comp_rep[comp] = (comp_rep['datetime'] - comp_rep[comp]) / np.timedelta64(1, 'D')

C:\Users\USER\anaconda3\envs\py38tf2\lib\site-packages\pandas\core\arrays\datetimelike.py:1108: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized
  warnings.warn(


In [7]:
# 통합 과정
final_feat = telemetry.merge(error_count, on=['datetime', 'machineID'], how='left')
final_feat = final_feat.merge(comp_rep, on=['datetime', 'machineID'], how='left')
final_feat = final_feat.merge(machines, on=['machineID'], how='left')

In [8]:
# failure 결합
labeled_features = final_feat.merge(failures, on=['datetime', 'machineID'], how='left')
# fillna 가 str일 경우 작동하지 않는 문제 발견 하여 comp 의 마지막 단어를 추출하여 fillna 진행후 comp를 다시 붙이는 방식으로 해결
labeled_features.failure = 'comp' + labeled_features.failure.str[-1].fillna(method='bfill', limit=23) # 앞선 23칸을 failure 로 설정
labeled_features.failure = labeled_features.failure.fillna('none') # 나머지를 none 으로 설정

In [9]:
labeled_features[73:100]

,datetime,machineID,volt,rotate,pressure,vibration,error1,error2,error3,error4,error5,comp1,comp2,comp3,comp4,model,age,failure
73,2015-01-04 07:00:00,1,142.666469,433.279499,118.853452,54.848731,0.0,0.0,0.0,0.0,0.0,22.041667,217.041667,157.041667,172.041667,model3,18,comp4
74,2015-01-04 08:00:00,1,191.168936,479.615136,101.999663,52.882567,0.0,0.0,0.0,0.0,0.0,22.083333,217.083333,157.083333,172.083333,model3,18,comp4
75,2015-01-04 09:00:00,1,157.436263,438.091311,113.100915,53.695544,0.0,0.0,0.0,0.0,0.0,22.125000,217.125000,157.125000,172.125000,model3,18,comp4
76,2015-01-04 10:00:00,1,153.143558,440.162685,94.524894,57.411078,0.0,0.0,0.0,0.0,0.0,22.166667,217.166667,157.166667,172.166667,model3,18,comp4
77,2015-01-04 11:00:00,1,215.656488,458.097746,95.036280,51.647981,0.0,0.0,0.0,0.0,0.0,22.208333,217.208333,157.208333,172.208333,model3,18,comp4
78,2015-01-04 12:00:00,1,173.525320,421.728389,100.617527,50.458297,0.0,0.0,0.0,0.0,0.0,22.250000,217.250000,157.250000,172.250000,model3,18,comp4
79,2015-01-04 13:00:00,1,169.501121,454.460114,91.675576,45.951349,0.0,0.0,0.0,0.0,0.0,22.291667,217.291667,157.291667,172.291667,model3,18,comp4
80,2015-01-04 14:00:00,1,129.016707,479.457721,111.575038,49.398412,0.0,0.0,0.0,0.0,0.0,22.333333,217.333333,157.333333,172.333333,model3,18,comp4
81,2015-01-04 15:00:00,1,168.503141,455.536868,83.689837,43.917862,0.0,0.0,0.0,0.0,0.0,22.375000,217.375000,157.375000,172.375000,model3,18,comp4
82,2015-01-04 16:00:00,1,184.640476,365.213804,87.474009,52.048152,0.0,0.0,0.0,0.0,0.0,22.416667,217.416667,157.416667,172.416667,model3,18,comp4


sclaer

In [10]:
# pick the feature columns 
scaler_cols = ['machineID', 'volt', 'rotate', 'pressure', 'vibration',
       'error1', 'error2', 'error3', 'error4', 'error5', 'comp1', 'comp2',
       'comp3', 'comp4', 'age']

In [11]:
scaler_dict = {}
for col_ in scaler_cols:
    scaler = StandardScaler()
    labeled_features[[col_]] = scaler.fit_transform(labeled_features[[col_]])
    scaler_dict[col_] = scaler

In [12]:
labeled_features[73:100]

,datetime,machineID,volt,rotate,pressure,vibration,error1,error2,error3,error4,error5,comp1,comp2,comp3,comp4,model,age,failure
73,2015-01-04 07:00:00,-1.714798,-1.812543,-0.252937,1.628559,2.693139,-0.033989,-0.0336,-0.030942,-0.028818,-0.020162,-0.503789,2.792392,1.771895,1.979790,model3,1.144539,comp4
74,2015-01-04 08:00:00,-1.714798,1.314742,0.626713,0.103228,2.327035,-0.033989,-0.0336,-0.030942,-0.028818,-0.020162,-0.503122,2.793095,1.772602,1.980488,model3,1.144539,comp4
75,2015-01-04 09:00:00,-1.714798,-0.860234,-0.161588,1.107932,2.478413,-0.033989,-0.0336,-0.030942,-0.028818,-0.020162,-0.502455,2.793798,1.773310,1.981186,model3,1.144539,comp4
76,2015-01-04 10:00:00,-1.714798,-1.137014,-0.122265,-0.573267,3.170254,-0.033989,-0.0336,-0.030942,-0.028818,-0.020162,-0.501789,2.794501,1.774018,1.981883,model3,1.144539,comp4
77,2015-01-04 11:00:00,-1.714798,2.893621,0.218220,-0.526985,2.097152,-0.033989,-0.0336,-0.030942,-0.028818,-0.020162,-0.501122,2.795204,1.774726,1.982581,model3,1.144539,comp4
78,2015-01-04 12:00:00,-1.714798,0.177137,-0.472227,-0.021861,1.875630,-0.033989,-0.0336,-0.030942,-0.028818,-0.020162,-0.500455,2.795907,1.775433,1.983279,model3,1.144539,comp4
79,2015-01-04 13:00:00,-1.714798,-0.082330,0.149162,-0.831141,1.036425,-0.033989,-0.0336,-0.030942,-0.028818,-0.020162,-0.499788,2.796610,1.776141,1.983977,model3,1.144539,comp4
80,2015-01-04 14:00:00,-1.714798,-2.692637,0.623724,0.969835,1.678277,-0.033989,-0.0336,-0.030942,-0.028818,-0.020162,-0.499122,2.797313,1.776849,1.984675,model3,1.144539,comp4
81,2015-01-04 15:00:00,-1.714798,-0.146677,0.169604,-1.553880,0.657785,-0.033989,-0.0336,-0.030942,-0.028818,-0.020162,-0.498455,2.798016,1.777557,1.985373,model3,1.144539,comp4
82,2015-01-04 16:00:00,-1.714798,0.893807,-1.545116,-1.211398,2.171665,-0.033989,-0.0336,-0.030942,-0.028818,-0.020162,-0.497788,2.798719,1.778264,1.986071,model3,1.144539,comp4


train test split

In [13]:
# make test and training splits
last_train_date, first_test_date = [pd.to_datetime('2015-07-31 01:00:00'), pd.to_datetime('2015-08-01 01:00:00')]

train_y = pd.get_dummies(labeled_features.loc[labeled_features['datetime'] < last_train_date, ['machineID','failure']])
train_X = pd.get_dummies(labeled_features.loc[labeled_features['datetime'] < last_train_date].drop(['failure'], 1))

test_y = pd.get_dummies(labeled_features.loc[labeled_features['datetime'] > first_test_date, ['machineID','failure']])
test_X = pd.get_dummies(labeled_features.loc[labeled_features['datetime'] > first_test_date].drop(['failure'], 1))

In [14]:
train_y.columns

Index(['machineID', 'failure_comp1', 'failure_comp2', 'failure_comp3',
       'failure_comp4', 'failure_none'],
      dtype='object')

Train

In [15]:
sequence_length = 50

In [16]:
# function to reshape features into (samples, time steps, features) 
def gen_sequence(id_df, seq_length, seq_cols):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_array[start:stop, :]

In [17]:
# pick the feature columns 
sequence_cols = ['machineID', 'volt', 'rotate', 'pressure', 'vibration',
       'error1', 'error2', 'error3', 'error4', 'error5', 'comp1', 'comp2',
       'comp3', 'comp4', 'age', 'model_model1', 'model_model2', 'model_model3',
       'model_model4']

In [18]:
# generator for the sequences
seq_gen = (list(gen_sequence(train_X[train_X['machineID']==id], sequence_length, sequence_cols)) 
           for id in train_X['machineID'].unique())

In [19]:
# generate sequences and convert to numpy array
seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
seq_array.shape

(500923, 50, 19)

In [20]:
# function to generate labels
def gen_labels(id_df, seq_length, label):
    data_array = id_df[label].values
    num_elements = data_array.shape[0]
    return data_array[seq_length:num_elements, :]

In [21]:
# generate labels
label_gen = [gen_labels(train_y[train_y['machineID']==id], sequence_length, ['failure_comp1', 'failure_comp2', 
                                                                            'failure_comp3','failure_comp4', 'failure_none']) 
             for id in train_y['machineID'].unique()]
label_array = np.concatenate(label_gen).astype(np.float32)
label_array.shape

(500923, 5)

In [22]:
# build the network
nb_features = seq_array.shape[2]
nb_out = label_array.shape[1]

model = Sequential()

model.add(LSTM(
         input_shape=(sequence_length, nb_features),
         units=100,
         return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(
          units=50,
          return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(units=nb_out, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [23]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50, 100)           48000     
_________________________________________________________________
dropout (Dropout)            (None, 50, 100)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense (Dense)                (None, 5)                 255       
Total params: 78,455
Trainable params: 78,455
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
%%time
# fit the network
model.fit(seq_array, label_array, epochs=10, batch_size=200, validation_split=0.05, verbose=1,
          callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')])

Epoch 1/10
2380/2380 [==============================] - 46s 16ms/step - loss: 0.0190 - accuracy: 0.9951 - val_loss: 0.0101 - val_accuracy: 0.9959
Wall time: 46.6 s


In [25]:
# training metrics
scores = model.evaluate(seq_array, label_array, verbose=1, batch_size=200)
print('Accurracy: {}'.format(scores[1]))

2505/2505 [==============================] - 21s 8ms/step - loss: 0.0051 - accuracy: 0.9981
Accurracy: 0.9980515837669373


In [26]:
# make predictions and compute confusion matrix
y_pred = model.predict_classes(seq_array,verbose=1, batch_size=200)
y_true = np.argmax(label_array, axis=1)
print('Confusion matrix\n- x-axis is true labels.\n- y-axis is predicted labels')
cm = confusion_matrix(y_true, y_pred)
cm

C:\Users\USER\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
2505/2505 [==============================] - 16s 6ms/step
Confusion matrix
- x-axis is true labels.
- y-axis is predicted labels


array([[  2496,      0,      0,     20,    244],
       [     0,   3227,      0,      1,     89],
       [     0,      0,   1711,      0,     19],
       [     1,      1,      3,   2146,     15],
       [   157,     19,    233,    174, 490367]], dtype=int64)

데이터 불균형으로 인한 에러